In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder


In [2]:
# Load the data. 
# Used zip file bcse csv file was too big to upload to github.
df = pd.read_csv('../Resources/2017_raw.csv')
df.head()

C:\Users\lmalla\Anaconda3.1\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (0,1,3,4,16) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,FL_DATE,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,...,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,Unnamed: 27
0,2/1/2017,UA,3.0,IAH,FLL,2000.0,1956.0,-4.0,18.0,2014.0,...,144.0,140.0,116.0,966.0,NaN,NaN,NaN,NaN,NaN,NaN
1,2/1/2017,UA,3.0,ORD,SNA,910.0,918.0,8.0,14.0,932.0,...,265.0,260.0,241.0,1726.0,NaN,NaN,NaN,NaN,NaN,NaN
2,2/1/2017,UA,3.0,SNA,IAH,1240.0,1604.0,204.0,10.0,1614.0,...,190.0,173.0,159.0,1346.0,187.0,0.0,0.0,0.0,0.0,NaN
3,2/1/2017,UA,10.0,AUS,DEN,600.0,547.0,-13.0,10.0,557.0,...,145.0,128.0,112.0,775.0,NaN,NaN,NaN,NaN,NaN,NaN
4,2/1/2017,UA,10.0,DEN,BIL,1115.0,1110.0,-5.0,7.0,1117.0,...,94.0,83.0,72.0,455.0,NaN,NaN,NaN,NaN,NaN,NaN
